In [1]:
!ls

download_and_save_images.ipynb ycup21_base.ipynb
images                         ycup21_base_gpu.ipynb
tokenizer.ipynb                ycup21_base_tpu.ipynb


In [2]:
!pwd

/Users/michaelka/Developer/ml/projects/ycup21/notebooks


In [3]:
!df -h

Filesystem       Size   Used  Avail Capacity iused      ifree %iused  Mounted on
/dev/disk3s1s1  228Gi   14Gi   24Gi    38%  553757 2393071203    0%   /
devfs           202Ki  202Ki    0Bi   100%     698          0  100%   /dev
/dev/disk3s6    228Gi  9.0Gi   24Gi    28%       9 2393624951    0%   /System/Volumes/VM
/dev/disk3s2    228Gi  219Mi   24Gi     1%     678 2393624282    0%   /System/Volumes/Preboot
/dev/disk3s4    228Gi  6.1Mi   24Gi     1%      45 2393624915    0%   /System/Volumes/Update
/dev/disk1s2    500Mi  6.0Mi  481Mi     2%       3    5119997    0%   /System/Volumes/xarts
/dev/disk1s1    500Mi  7.5Mi  481Mi     2%      34    5119966    0%   /System/Volumes/iSCPreboot
/dev/disk1s3    500Mi  336Ki  481Mi     1%      34    5119966    0%   /System/Volumes/Hardware
/dev/disk3s5    228Gi  180Gi   24Gi    89%  978159 2392646801    0%   /System/Volumes/Data
map auto_home     0Bi    0Bi    0Bi   100%       0          0  100%   /System/Volumes/Data/home
/dev/disk3s3    228Gi  96

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!wget https://cvlab.s3.yandex.net/mlcup2021/images.json

--2021-10-08 17:01:14--  https://cvlab.s3.yandex.net/mlcup2021/images.json
Resolving cvlab.s3.yandex.net (cvlab.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to cvlab.s3.yandex.net (cvlab.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 605923636 (578M) [application/json]
Saving to: ‘images.json’

images.json         100%[===================>] 577.85M  22.9MB/s    in 26s     

2021-10-08 17:01:41 (22.0 MB/s) - ‘images.json’ saved [605923636/605923636]



In [1]:
BASE_PATH = '/content/drive/MyDrive/ai/ycup21/'

In [2]:
!ls {BASE_PATH}

ls: /content/drive/MyDrive/ai/ycup21/: No such file or directory


In [3]:
BASE_PATH_DATA = f'/content/drive/MyDrive/ai/ycup21/data'

In [4]:
# !wget https://cvlab.s3.yandex.net/mlcup2021/metadata.json

In [5]:
# !cp ./metadata.json {BASE_PATH_DATA}

In [15]:
import json
import os

import requests
from io import BytesIO
from PIL import Image

In [7]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [8]:
import warnings

warnings.filterwarnings("ignore")

In [17]:
data_path = '../data'
# data_path = './'

image_save_path = './images'
image_size = 512

In [18]:
!mkdir images

mkdir: images: File exists


In [11]:
with open(os.path.join(data_path, 'images.json')) as json_file:
    json_strings = json_file.readlines()

In [12]:
len(json_strings)

5462418

In [13]:
for json_string in json_strings[:10]:
    image_data = json.loads(json_string)
    print(image_data['image'], image_data['url'])

1 http://static.pleer.ru/i/gp/826/344/norm.jpg
2 https://i0.wp.com/allwomens.ru/uploads/posts/2018-01/1515670311_mariya-gorban-10.jpg
3 https://99px.ru/sstorage/56/2015/10/image_561810151141106183724.png
4 https://grandmirage.ru/5158-large_default/stol-pismennyj-fierta-43-021.jpg
5 https://wallpaper-mania.com/wp-content/uploads/2018/09/High_resolution_wallpaper_background_ID_77700295631.jpg
6 https://drasler.ru/wp-content/uploads/2019/08/%D0%94%D0%BE%D0%B1%D1%80%D0%BE%D0%B5-%D1%83%D1%82%D1%80%D0%BE-%D0%BA%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8-%D0%B4%D0%BB%D1%8F-%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%D1%8B-%D1%81-%D0%BD%D0%B0%D0%B4%D0%BF%D0%B8%D1%81%D1%8F%D0%BC%D0%B8-%D0%BA%D0%BE%D1%84%D0%B5-30.jpg
7 https://static.wikia.nocookie.net/fantasticbeings/images/d/d2/%D0%AE%D1%81%D1%82%D1%80%D0%B8%D1%86%D0%B0.jpg/revision/latest?cb=20200408081506&path-prefix=ru
8 https://img-fotki.yandex.ru/get/6512/20839000.3e/0_85e9c_8d598366_orig
9 https://avatars.mds.yandex.net/get-zen_doc/2816669/pub_5

In [14]:
json_strings[:10]

['{"image": 1, "url": "http://static.pleer.ru/i/gp/826/344/norm.jpg"}\n',
 '{"image": 2, "url": "https://i0.wp.com/allwomens.ru/uploads/posts/2018-01/1515670311_mariya-gorban-10.jpg"}\n',
 '{"image": 3, "url": "https://99px.ru/sstorage/56/2015/10/image_561810151141106183724.png"}\n',
 '{"image": 4, "url": "https://grandmirage.ru/5158-large_default/stol-pismennyj-fierta-43-021.jpg"}\n',
 '{"image": 5, "url": "https://wallpaper-mania.com/wp-content/uploads/2018/09/High_resolution_wallpaper_background_ID_77700295631.jpg"}\n',
 '{"image": 6, "url": "https://drasler.ru/wp-content/uploads/2019/08/%D0%94%D0%BE%D0%B1%D1%80%D0%BE%D0%B5-%D1%83%D1%82%D1%80%D0%BE-%D0%BA%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8-%D0%B4%D0%BB%D1%8F-%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%D1%8B-%D1%81-%D0%BD%D0%B0%D0%B4%D0%BF%D0%B8%D1%81%D1%8F%D0%BC%D0%B8-%D0%BA%D0%BE%D1%84%D0%B5-30.jpg"}\n',
 '{"image": 7, "url": "https://static.wikia.nocookie.net/fantasticbeings/images/d/d2/%D0%AE%D1%81%D1%82%D1%80%D0%B8%D1%86%D0%B0.j

In [12]:
!pip install jsonlines

You should consider upgrading via the '/Users/michaelka/Developer/ml/projects/p3env/bin/python -m pip install --upgrade pip' command.


In [13]:
import jsonlines
from tqdm.auto import tqdm

In [19]:
image_data = []
tqdm_load = True

with jsonlines.open(os.path.join(data_path, 'images.json')) as reader:
    if tqdm_load:
        reader = tqdm(reader)
    for obj in reader:
        image_data.append((obj['image'], obj['url']))

0it [00:00, ?it/s]

In [20]:
image_data[:10]

[(1, 'http://static.pleer.ru/i/gp/826/344/norm.jpg'),
 (2,
  'https://i0.wp.com/allwomens.ru/uploads/posts/2018-01/1515670311_mariya-gorban-10.jpg'),
 (3, 'https://99px.ru/sstorage/56/2015/10/image_561810151141106183724.png'),
 (4,
  'https://grandmirage.ru/5158-large_default/stol-pismennyj-fierta-43-021.jpg'),
 (5,
  'https://wallpaper-mania.com/wp-content/uploads/2018/09/High_resolution_wallpaper_background_ID_77700295631.jpg'),
 (6,
  'https://drasler.ru/wp-content/uploads/2019/08/%D0%94%D0%BE%D0%B1%D1%80%D0%BE%D0%B5-%D1%83%D1%82%D1%80%D0%BE-%D0%BA%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8-%D0%B4%D0%BB%D1%8F-%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%D1%8B-%D1%81-%D0%BD%D0%B0%D0%B4%D0%BF%D0%B8%D1%81%D1%8F%D0%BC%D0%B8-%D0%BA%D0%BE%D1%84%D0%B5-30.jpg'),
 (7,
  'https://static.wikia.nocookie.net/fantasticbeings/images/d/d2/%D0%AE%D1%81%D1%82%D1%80%D0%B8%D1%86%D0%B0.jpg/revision/latest?cb=20200408081506&path-prefix=ru'),
 (8, 'https://img-fotki.yandex.ru/get/6512/20839000.3e/0_85e9c_8d598366_

# asyncio

In [1]:
import asyncio

In [3]:
async def foo():
    print('Running in foo')
    await asyncio.sleep(0)
    print('Explicit context switch to foo again')


async def bar():
    print('Explicit context to bar')
    await asyncio.sleep(0)
    print('Implicit context switch back to bar')

In [5]:
async def main():
    await asyncio.gather(foo(), bar())

In [6]:
await main()

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


# 1

In [16]:
!pip install aiohttp
!pip install aiofiles

You should consider upgrading via the '/Users/michaelka/Developer/ml/projects/p3env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/michaelka/Developer/ml/projects/p3env/bin/python -m pip install --upgrade pip' command.


In [22]:
import aiohttp
import aiofiles
import asyncio

In [19]:
image_json = json.loads(json_strings[0])
image_json['url']

'http://static.pleer.ru/i/gp/826/344/norm.jpg'

In [41]:
single_image = image_json['url']

async with aiohttp.ClientSession() as session:
    async with session.get(single_image) as resp:
        
        single_image_name = single_image.split('/')[-1]

        if resp.status == 200:
            async with aiofiles.open(single_image_name, mode='wb') as f:
                await f.write(await resp.read())
                await f.close()

In [29]:
image_formats = ("image/png", "image/jpeg", "image/jpg")

def is_url_image(r):
    if "content-type" in r.headers and r.headers["content-type"] in image_formats:
        return True
    return False

In [30]:
async def process_image(image_bytes):
    image = Image.open(BytesIO(image_bytes))
    image = image.convert('RGB')
    # image = image.resize((image_size, image_size))
    image.thumbnail((image_size, image_size))
    return image

In [65]:
timeout_seconds = 10
timeout = aiohttp.ClientTimeout(total=None,sock_connect=timeout_seconds,sock_read=timeout_seconds)

In [66]:
async def fetch_async(image_url, image_name):
    print('Fetch async: ', image_name)
    
    try:
        async with aiohttp.request('GET', image_url, timeout=timeout) as response:

            if response.status != 200:
                print('Image couldn\'t be retreived: ', image_name, response.status)
                return

            if not is_url_image(response):
                print('Not an image: ', image_name)
                return

            print('Image sucessfully downloaded: ', image_name)

            image_bytes = await response.read()

            image = await process_image(image_bytes)

            img_byte_arr = BytesIO()
            image.save(img_byte_arr, format='jpeg')
            img_byte_arr = img_byte_arr.getvalue()

            image_path = os.path.join(image_save_path, image_name)
            async with aiofiles.open(image_path, mode='wb') as f:
                await f.write(img_byte_arr)
                await f.close()
    except asyncio.TimeoutError as te:
        print('TimeoutError: ', str(te))
    except aiohttp.ClientError as e:
        print('Exception: ', str(e))

In [67]:
async def main(image_data):
    tasks = []
    
    for image in image_data:
                
        image_id = image[0]
        image_name = f'image_{image_id}.jpg'
        image_url = image[1]

        tasks.append(fetch_async(image_url, image_name))
    
    
    await asyncio.gather(*tasks)
    print('loading finished')

In [68]:
await main(image_data[:100])

Fetch async:  image_1.jpg
Fetch async:  image_2.jpg
Fetch async:  image_3.jpg
Fetch async:  image_4.jpg
Fetch async:  image_5.jpg
Fetch async:  image_6.jpg
Fetch async:  image_7.jpg
Fetch async:  image_8.jpg
Fetch async:  image_9.jpg
Fetch async:  image_10.jpg
Fetch async:  image_11.jpg
Fetch async:  image_12.jpg
Fetch async:  image_13.jpg
Fetch async:  image_14.jpg
Fetch async:  image_15.jpg
Fetch async:  image_16.jpg
Fetch async:  image_17.jpg
Fetch async:  image_18.jpg
Fetch async:  image_19.jpg
Fetch async:  image_21.jpg
Fetch async:  image_22.jpg
Fetch async:  image_24.jpg
Fetch async:  image_25.jpg
Fetch async:  image_26.jpg
Fetch async:  image_27.jpg
Fetch async:  image_28.jpg
Fetch async:  image_29.jpg
Fetch async:  image_30.jpg
Fetch async:  image_31.jpg
Fetch async:  image_32.jpg
Fetch async:  image_33.jpg
Fetch async:  image_34.jpg
Fetch async:  image_35.jpg
Fetch async:  image_36.jpg
Fetch async:  image_37.jpg
Fetch async:  image_39.jpg
Fetch async:  image_40.jpg
Fetch asyn

# 2

In [58]:
async def fetch(session, image_url, image_name):
    try:     
        async with session.get(image_url) as resp:

            image_path = os.path.join(image_save_path, image_name)

            print(image_url, image_name)
    #         print(resp.headers)

            if resp.status != 200:
                print('Image couldn\'t be retreived: ', resp.status)
                return

            if not is_url_image(resp):
                print('Not an image: ', image_name)
                return

            image_bytes = await resp.read()
            image = await process_image(image_bytes)

            img_byte_arr = BytesIO()
            image.save(img_byte_arr, format='jpeg')
            img_byte_arr = img_byte_arr.getvalue()

            async with aiofiles.open(image_path, mode='wb') as f:
                await f.write(img_byte_arr)
                await f.close()
    except Exception as e:
          print('Exception', str(e))    

In [14]:
index_to_remove = []
id_to_remove = []

In [59]:
timeout = aiohttp.ClientTimeout(total=1)
connector = aiohttp.TCPConnector(verify_ssl=False)

In [15]:
start_from = 26617
num_images_to_load = 100000

In [21]:
from tqdm import tqdm

In [28]:
t0 = time.time()

num_of_images = 0

index_to_remove = []
id_to_remove = []

# pbar = tqdm(enumerate(json_strings[start_from:num_images_to_load]))

for index, json_string in enumerate(json_strings[start_from:num_images_to_load]):
    
#     if index in index_to_remove:
#         continue
    
    t1 = time.time()
    
    image_json = json.loads(json_string)
    
#     print(index, image_json['image'], image_json['url'])
      
    image_id = image_json['image']
    image_name = f'image_{image_id}.jpg'
      
    try:
        r = requests.get(image_json['url'], verify=False, timeout=1)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        # print('Image couldn\'t be retreived: ', index, image_name, e, format_time(time.time() - t1))
        index_to_remove.append(index)
        id_to_remove.append(image_id)
        continue

    if r.status_code != 200:
        print('Image couldn\'t be retreived: ', index, image_name, r.status_code, format_time(time.time() - t1))
        index_to_remove.append(index)
        id_to_remove.append(image_id)
        continue

    if not is_url_image(r):
        print('Not an image: ', index, image_name, r.status_code, format_time(time.time() - t1))
        index_to_remove.append(index)
        id_to_remove.append(image_id)
        continue
       

    # print('Image sucessfully downloaded: ', index, image_name, format_time(time.time() - t1))
    # with open(os.path.join(image_save_path, image_name), 'wb') as handler:
    #     handler.write(r.content)

    num_of_images += 1
    time_since_b = time.time() - t0
    # print(time.time() - t0)
    # pbar.set_description
    print(f'downloaded images: {num_of_images} [{num_of_images/time_since_b}im/s]')

    image = Image.open(BytesIO(r.content))
    image = image.convert('RGB')
    # image = image.resize((image_size, image_size))
    image.thumbnail((image_size, image_size))
    image.save(os.path.join(image_save_path, image_name))
           
print('loading finished for: {}'.format(format_time(time.time() - t0)))

downloaded images: 1 [0.9032998636964599im/s]
downloaded images: 2 [0.26447241514291825im/s]
downloaded images: 3 [0.3903028405827095im/s]
downloaded images: 4 [0.44381996003317403im/s]
downloaded images: 5 [0.49267364923586615im/s]
downloaded images: 6 [0.545241671513883im/s]
downloaded images: 7 [0.5682274124443467im/s]
downloaded images: 8 [0.6036024785771009im/s]
downloaded images: 9 [0.6243963270636936im/s]
downloaded images: 10 [0.6626992806315556im/s]
downloaded images: 11 [0.6953987275671262im/s]
downloaded images: 12 [0.7080652120635111im/s]
Image couldn't be retreived:  12 image_28979.jpg 406 0:00:00
downloaded images: 13 [0.7502259754009153im/s]
downloaded images: 14 [0.7826782228318105im/s]
downloaded images: 15 [0.8049578534477713im/s]
downloaded images: 16 [0.8398997875334209im/s]
downloaded images: 17 [0.8300395208144465im/s]
downloaded images: 18 [0.8390498006289795im/s]
downloaded images: 19 [0.8217623052141146im/s]
downloaded images: 20 [0.8533092454716371im/s]
downlo

KeyboardInterrupt: ignored

In [ ]:
# index_to_remove, id_to_remove
id_to_remove[-10:], index_to_remove[-10:]

([28841, 28850, 28852, 28864, 28908, 28919, 28954, 28959, 28960, 28965],
 [26499, 26507, 26509, 26519, 26561, 26572, 26605, 26609, 26610, 26615])

In [ ]:
import numpy as np

In [ ]:
np.save('index_to_remove.npy', index_to_remove)
np.save('id_to_remove.npy', id_to_remove)

In [ ]:
len(index_to_remove), len(id_to_remove)

(1990, 1990)

In [ ]:
# 1_00 - 1MB
# 1_000_000 - 10GB

In [26]:
async def main(image_data):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for image in image_data:
            
            image_id = image[0]
            image_name = f'image_{image_id}.jpg'
            image_url = image[1]
            
            image_path = os.path.join(image_save_path, image_name)
            
            async with session.get(image_url) as resp:
                if resp.status == 200:
                    async with aiofiles.open(image_path, mode='wb') as f:
                        await f.write(await resp.read())
                        await f.close()

In [ ]:
# 12 13 402 error
# 19 21 SSLCertVerificationError https://stackoverflow.com/questions/10667960/python-requests-throwing-sslerror
# 40 44 ConnectionResetError
#

In [ ]:
json_string = json_strings[26616]
image_json = json.loads(json_string)
print(image_json['image'], image_json['url'])

28966 http://www.cinemapassion.com/stickers4/Oggy_et_les_cafards_Le_film-21091115122013.jpg


In [ ]:
try:
    r = requests.get(image_json['url'], verify=False, timeout=2)
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print(e)

In [45]:
asyncio.run(main(image_data[:10]))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
is_url_image(r)

True

In [ ]:
r.status_code

200

In [ ]:
# image = Image.open(requests.get(image_json['url'], verify=False, stream=True).raw)
image = Image.open(BytesIO(r.content))
image = image.convert('RGB')
# image = image.resize((image_size, image_size))
image.thumbnail((image_size, image_size))

In [ ]:
np.array(image).shape

(509, 512, 3)

In [ ]:
# image

In [ ]:
size = 256
# resample = None
im = image.resize((size, size))

In [ ]:
im.save(os.path.join(image_save_path, image_name))

# Create archive

In [ ]:
!tar -czf ycup21-images-1.tgz {image_save_path}

In [ ]:
!ls -lh ./ycup21-images-1.tgz

-rw-r--r-- 1 root root 705M Oct  4 17:29 ./ycup21-images-1.tgz


In [ ]:
!ls

drive		  images       index_to_remove.npy  ycup21-images-1.tgz
id_to_remove.npy  images.json  sample_data


In [ ]:
!cp ./ycup21-images-1.tgz {BASE_PATH_DATA}

In [ ]:
!cp ./index_to_remove.npy {BASE_PATH_DATA}

In [ ]:
!cp ./id_to_remove.npy {BASE_PATH_DATA}

In [ ]:
images = os.listdir(image_save_path)
len(images)

24627

In [ ]:
len(images) + len(index_to_remove)

26617